In [1]:
import pandas as pd
import numpy as np

In [31]:
df = pd.read_csv('data/1983.csv')

In [3]:
df[df['day']==1]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),True,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),False
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),False,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),True
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4605,1983.11,1,4107,M3w,Kaiki,0-1 (4-11),False,yorikiri,4080,Y2eHD,Kitanoumi,1-0 (11-4),True
4606,1983.11,1,1350,M3e,Onokuni,1-0 (10-5),True,yoritaoshi,1354,Y1w,Chiyonofuji,0-1 (14-1),False
4607,1983.11,1,1354,Y1w,Chiyonofuji,0-1 (14-1),False,yoritaoshi,1350,M3e,Onokuni,1-0 (10-5),True
4608,1983.11,1,1342,K1w,Asahifuji,0-1 (6-9),False,yorikiri,4104,Y1e,Takanosato,1-0 (13-2),True


In [4]:
TEST_RECORDS = {
    0:'1-0 (8-7)',
    1:'(12-3)',
    2:'0-9 (4-11)',
    3:'12-2 (13-2)',
    4:'3-12 (3-12)',
    5:'- (-)',
    6:'3-2',
    7:'10-1-2',
    8:'3-1 (3-1-12)',
    9:'2-10-1 (2-10-3)',
}

In [5]:
def absence_detect(record):
    if not record.find('-') == record.rfind('-'):
        return True
    return False

In [6]:
def test_absence():
    assert(absence_detect(TEST_RECORDS[7]) == True)
    assert(absence_detect(TEST_RECORDS[6]) == False)
test_absence()

In [7]:
def get_start_index(record):
    parentheses_index = record.find('(')
    if parentheses_index == 0:
        return 1
    return 0

In [8]:
def get_first_index(record):
    return record.find('-')

In [9]:
def get_last_index(record):
    return record.rfind('-')

In [10]:
def get_end_index(record):
    open_par_index = record.find('(')
    if open_par_index and open_par_index != -1:
        return open_par_index - 1
    close_par_index = record.find(')') #Only happens if opening parenthesis at index 0
    if close_par_index and close_par_index != -1:
        return close_par_index
    return len(record)

In [11]:
def test_indexes():
    assert(get_start_index(TEST_RECORDS[0]) == 0)
    assert(get_start_index(TEST_RECORDS[1]) == 1)
    assert(get_first_index(TEST_RECORDS[6]) == get_last_index(TEST_RECORDS[6]) == 1)
    assert(get_first_index(TEST_RECORDS[7]) == 2)
    assert(get_last_index(TEST_RECORDS[7]) == 4)
    assert(get_end_index(TEST_RECORDS[0]) == 3)
    assert(get_end_index(TEST_RECORDS[1]) == 5)
    assert(get_end_index(TEST_RECORDS[6]) == 3)
    assert(get_end_index(TEST_RECORDS[7]) == 6)
    assert(get_end_index(TEST_RECORDS[8]) == 3)
    assert(get_end_index(TEST_RECORDS[9]) == 6)
test_indexes()

In [12]:
def get_post_fight(record):
    start = get_start_index(record)
    end = get_end_index(record)
    post_fight = record[start:end]
    return post_fight

In [13]:
def test_post_fight():
    assert(get_post_fight(TEST_RECORDS[0]) == '1-0')
    assert(get_post_fight(TEST_RECORDS[1]) == '12-3')
    assert(get_post_fight(TEST_RECORDS[2]) == '0-9')
    assert(get_post_fight(TEST_RECORDS[3]) == '12-2')
    assert(get_post_fight(TEST_RECORDS[4]) == '3-12')
    assert(get_post_fight(TEST_RECORDS[6]) == '3-2')
    assert(get_post_fight(TEST_RECORDS[7]) == '10-1-2')
    assert(get_post_fight(TEST_RECORDS[8]) == '3-1')
    assert(get_post_fight(TEST_RECORDS[9]) == '2-10-1')
test_post_fight()

In [14]:
import functools

def nan_exception(func):                                                                         
    """ Decorator to return np.nan when exception is raised | func --> func """                                                                                               
    @functools.wraps(func)                                                                            
    def wrapper(*args, **kwargs):                                                                     
        try:                                                                                      
            wrapped_func = func(*args, **kwargs)                                                  
        except Exception as e:                                                                    
            wrapped_func = np.nan                                                                                                                          
        return wrapped_func                                                                           
    return wrapper

In [15]:
@nan_exception
def get_post_wins(record):
    record = get_post_fight(record)
    end_index = get_first_index(record)
    return int(record[:end_index])

In [16]:
def test_post_wins():
    assert(get_post_wins(TEST_RECORDS[0]) == 1)
    assert(get_post_wins(TEST_RECORDS[1]) == 12)
    assert(get_post_wins(TEST_RECORDS[2]) == 0)
    assert(get_post_wins(TEST_RECORDS[3]) == 12)
    assert(get_post_wins(TEST_RECORDS[4]) == 3)
    assert(np.isnan(get_post_wins(TEST_RECORDS[5])))
    assert(get_post_wins(TEST_RECORDS[6]) == 3)
    assert(get_post_wins(TEST_RECORDS[7]) == 10)
test_post_wins()

In [17]:
@nan_exception
def get_post_losses(record):
    record = get_post_fight(record)
    if absence_detect(record):
        start_index = get_first_index(record) + 1
        end_index = get_last_index(record)
        return int(record[start_index:end_index])
    start_index = get_first_index(record) + 1
    return int(record[start_index:])

In [18]:
def test_post_losses():
    assert(get_post_losses(TEST_RECORDS[0]) == 0)
    assert(get_post_losses(TEST_RECORDS[1]) == 3)
    assert(get_post_losses(TEST_RECORDS[2]) == 9)
    assert(get_post_losses(TEST_RECORDS[3]) == 2)
    assert(get_post_losses(TEST_RECORDS[4]) == 12)
    assert(np.isnan(get_post_losses(TEST_RECORDS[5])))
    assert(get_post_losses(TEST_RECORDS[6]) == 2)
    assert(get_post_losses(TEST_RECORDS[7]) == 1)
test_post_losses()

In [19]:
@nan_exception
def get_post_absences(record):
    record = get_post_fight(record)
    if absence_detect(record):
        start_index = get_last_index(record) + 1
        return int(record[start_index:])
    return 0

In [20]:
def test_post_absences():
    assert(get_post_absences(TEST_RECORDS[6]) == 0)
    assert(get_post_absences(TEST_RECORDS[7]) == 2)
    assert(get_post_absences(get_post_fight(TEST_RECORDS[8])) == 1)
    assert(get_post_absences(get_post_fight(TEST_RECORDS[9])) == 1)

In [21]:
df

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),True,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),False
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),False,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),True
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,1983.11,15,4104,Y1e,Takanosato,13-2,False,yorikiri,1354,Y1w,Chiyonofuji,14-1,True
5484,1983.11,16,1348,J4w,Jingaku,(11-4),False,sotogake,1378,J1w,Hoo,(11-4),True
5485,1983.11,16,1378,J1w,Hoo,(11-4),True,sotogake,1348,J4w,Jingaku,(11-4),False
5486,1983.11,16,1287,J12w,Konishiki,(11-4),False,yorikiri,1378,J1w,Hoo,(11-4),True


In [22]:
def convert_match(match):
    ref = match.copy()
    ref['rikishi1_id'] = match['rikishi2_id']
    ref['rikishi2_id'] = match['rikishi1_id']
    ref['rikishi1_win'] = match['rikishi2_win']
    ref['rikishi2_win'] = match['rikishi1_win']
    ref['rikishi1_result'] = match['rikishi2_result']
    ref['rikishi2_result'] = match['rikishi1_result']
    ref['rikishi1_shikona'] = match['rikishi2_shikona']
    ref['rikishi2_shikona'] = match['rikishi1_shikona']
    ref['rikishi1_rank'] = match['rikishi2_rank']
    ref['rikishi2_rank'] = match['rikishi1_rank']
    return ref

In [23]:
def same_match(match1, match2):
    ref = convert_match(match2)
    bool_series = match1 == ref
    return bool_series.all()

In [40]:
df = pd.read_csv('data/1983.csv')
df2 = df.shift(fill_value=0)
df2

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,0.00,0,0,0,0,0,0,0,0,0,0,0,0
1,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True
2,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),True,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),False
3,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False
4,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),False,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,1983.11,15,1354,Y1w,Chiyonofuji,14-1,True,yorikiri,4104,Y1e,Takanosato,13-2,False
5484,1983.11,15,4104,Y1e,Takanosato,13-2,False,yorikiri,1354,Y1w,Chiyonofuji,14-1,True
5485,1983.11,16,1348,J4w,Jingaku,(11-4),False,sotogake,1378,J1w,Hoo,(11-4),True
5486,1983.11,16,1378,J1w,Hoo,(11-4),True,sotogake,1348,J4w,Jingaku,(11-4),False


In [41]:
df3 = df2.apply(convert_match, axis=1)
compare_previous = (df == df3).all(axis=1)
compare_previous
df.loc[:,'compare_previous'] = compare_previous
df

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win,compare_previous
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True,False
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),True,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),False,True
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False,False
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),False,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),True,True
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,1983.11,15,4104,Y1e,Takanosato,13-2,False,yorikiri,1354,Y1w,Chiyonofuji,14-1,True,True
5484,1983.11,16,1348,J4w,Jingaku,(11-4),False,sotogake,1378,J1w,Hoo,(11-4),True,False
5485,1983.11,16,1378,J1w,Hoo,(11-4),True,sotogake,1348,J4w,Jingaku,(11-4),False,True
5486,1983.11,16,1287,J12w,Konishiki,(11-4),False,yorikiri,1378,J1w,Hoo,(11-4),True,False


In [42]:
unique = df.loc[~df.compare_previous, :'rikishi2_win']
unique

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True
6,1983.01,1,4109,J10w,Kotogatake,0-1 (7-8),False,tsukidashi,4129,J11e,Hakuryuyama,1-0 (3-12),True
8,1983.01,1,4111,J10e,Zaonishiki,0-1 (4-11),False,hatakikomi,4130,J9w,Hachiya,1-0 (8-7),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,1983.11,15,1363,O2eHD,Hokutenyu,11-4,True,yorikiri,4112,O1w,Kotokaze,11-4,False
5480,1983.11,15,4080,Y2eHD,Kitanoumi,11-4,False,yorikiri,4127,O1e,Wakashimazu,11-4,True
5482,1983.11,15,1354,Y1w,Chiyonofuji,14-1,True,yorikiri,4104,Y1e,Takanosato,13-2,False
5484,1983.11,16,1348,J4w,Jingaku,(11-4),False,sotogake,1378,J1w,Hoo,(11-4),True


In [44]:
unique.loc[:,'post1_wins'] = unique.loc[:,'rikishi1_result'].apply(get_post_wins)
unique.loc[:,'post1_losses'] = unique.loc[:,'rikishi1_result'].apply(get_post_losses)

In [45]:
unique.loc[:,'post2_wins'] = unique.loc[:,'rikishi2_result'].apply(get_post_wins)
unique.loc[:,'post2_losses'] = unique.loc[:,'rikishi2_result'].apply(get_post_losses)

In [47]:
unique.loc[unique.isnull()['post1_losses'],:]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win,post2_absences,post1_absences,post1_wins,post1_losses,post2_wins,post2_losses


In [48]:
unique.loc[unique.isnull().any(axis=1),unique.isnull().any(axis=0)]

""


In [49]:
unique.loc[unique.isnull().any(axis=1),:]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win,post2_absences,post1_absences,post1_wins,post1_losses,post2_wins,post2_losses


In [50]:
unique.loc[:,'post2_absences'] = unique.loc[:,'rikishi2_result'].apply(get_post_absences)
unique.loc[:,'post1_absences'] = unique.loc[:,'rikishi1_result'].apply(get_post_absences)

In [51]:
unique.loc[unique.isnull().any(axis=1),unique.isnull().any(axis=0)]

""


In [52]:
unique.loc[unique.isnull().any(axis=1),:]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win,post2_absences,post1_absences,post1_wins,post1_losses,post2_wins,post2_losses


In [53]:
unique

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win,post2_absences,post1_absences,post1_wins,post1_losses,post2_wins,post2_losses
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True,0,0,0,1,1,0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False,0,0,1,0,0,1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True,0,0,0,1,1,0
6,1983.01,1,4109,J10w,Kotogatake,0-1 (7-8),False,tsukidashi,4129,J11e,Hakuryuyama,1-0 (3-12),True,0,0,0,1,1,0
8,1983.01,1,4111,J10e,Zaonishiki,0-1 (4-11),False,hatakikomi,4130,J9w,Hachiya,1-0 (8-7),True,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,1983.11,15,1363,O2eHD,Hokutenyu,11-4,True,yorikiri,4112,O1w,Kotokaze,11-4,False,0,0,11,4,11,4
5480,1983.11,15,4080,Y2eHD,Kitanoumi,11-4,False,yorikiri,4127,O1e,Wakashimazu,11-4,True,0,0,11,4,11,4
5482,1983.11,15,1354,Y1w,Chiyonofuji,14-1,True,yorikiri,4104,Y1e,Takanosato,13-2,False,0,0,14,1,13,2
5484,1983.11,16,1348,J4w,Jingaku,(11-4),False,sotogake,1378,J1w,Hoo,(11-4),True,0,0,11,4,11,4


In [70]:
unique.loc[:,'pre1_wins'] = unique.loc[:,'post1_wins'] - unique.loc[:,'rikishi1_win']
unique.loc[:,'pre2_wins'] = unique.loc[:,'post2_wins'] - unique.loc[:,'rikishi2_win']
unique.loc[:,'pre1_losses'] = unique.loc[:,'post1_losses'] - unique.loc[:,'rikishi2_win']
unique.loc[:,'pre2_losses'] = unique.loc[:,'post2_losses'] - unique.loc[:,'rikishi1_win']
unique.loc[unique.day == 16, 'pre1_wins'] = test.loc[unique.day == 16, 'post1_wins'] #Playoff adjustment - playoff wins don't count in records
unique.loc[unique.day == 16, 'pre2_wins'] = test.loc[unique.day == 16, 'post2_wins']
unique.loc[unique.day == 16, 'pre1_losses'] = test.loc[unique.day == 16, 'post1_losses']
unique.loc[unique.day == 16, 'pre2_losses'] = test.loc[unique.day == 16, 'post2_losses']
unique.loc[unique.day == 16, :]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,...,post2_absences,post1_absences,post1_wins,post1_losses,post2_wins,post2_losses,pre1_wins,pre2_wins,pre1_losses,pre2_losses
922,1983.01,16,1347,J6e,Itai,(11-4),False,oshitaoshi,1350,J2w,...,0,0,11,4,11,4,11,11,4,4
924,1983.01,16,1350,J2w,Onokuni,(11-4),True,yoritaoshi,1380,J3w,...,0,0,11,4,11,4,11,11,4,4
926,1983.01,16,1379,S1w,Asashio,(14-1),False,yorikiri,4112,O1w,...,0,0,14,1,14,1,14,14,1,1
2696,1983.05,16,1451,J12e,Tochiizumi,(11-4),True,yoritaoshi,4132,J9w,...,0,0,11,4,11,4,11,11,4,4
3636,1983.07,16,1339,J5e,Hoshi,(10-5),True,yorikiri,1364,J13w,...,0,0,10,5,10,5,10,10,5,5
5484,1983.11,16,1348,J4w,Jingaku,(11-4),False,sotogake,1378,J1w,...,0,0,11,4,11,4,11,11,4,4
5486,1983.11,16,1287,J12w,Konishiki,(11-4),False,yorikiri,1378,J1w,...,0,0,11,4,11,4,11,11,4,4


In [93]:
short = unique[['rikishi1_result', 'rikishi2_result', 'rikishi1_win', 'rikishi2_win', 'pre1_wins', 'pre1_losses', 'post1_wins', 'post1_losses', 'pre2_wins', 'pre2_losses', 'post2_wins', 'post2_losses']]

In [94]:
short

,rikishi1_result,rikishi2_result,rikishi1_win,rikishi2_win,pre1_wins,pre1_losses,post1_wins,post1_losses,pre2_wins,pre2_losses,post2_wins,post2_losses
0,0-1 (7-8),1-0 (6-1),False,True,0,0,0,1,0,0,1,0
2,1-0 (9-6),0-1 (3-12),True,False,0,0,1,0,0,0,0,1
4,0-1 (8-7),1-0 (5-10),False,True,0,0,0,1,0,0,1,0
6,0-1 (7-8),1-0 (3-12),False,True,0,0,0,1,0,0,1,0
8,0-1 (4-11),1-0 (8-7),False,True,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5478,11-4,11-4,True,False,10,4,11,4,11,3,11,4
5480,11-4,11-4,False,True,11,3,11,4,10,4,11,4
5482,14-1,13-2,True,False,13,1,14,1,13,1,13,2
5484,(11-4),(11-4),False,True,11,4,11,4,11,4,11,4


In [95]:
short1 = short[['pre1_wins', 'pre1_losses', 'rikishi1_win']]

In [96]:
short2 = short[['pre2_wins', 'pre2_losses', 'rikishi2_win']]

In [97]:
combine1 = short1.rename(columns={'pre1_wins':'wins','pre1_losses':'losses','rikishi1_win':'victory'})
combine2 = short2.rename(columns={'pre2_wins':'wins','pre2_losses':'losses','rikishi2_win':'victory'})

In [98]:
combine = combine1.append(combine2)

In [103]:
rikishi_wl = combine.reset_index()

In [107]:
rikishi_wl

,index,wins,losses,victory
0,0,0,0,False
1,2,0,0,True
2,4,0,0,False
3,6,0,0,False
4,8,0,0,False
...,...,...,...,...
5483,5478,11,3,False
5484,5480,10,4,True
5485,5482,13,1,False
5486,5484,11,4,True


In [108]:
rikishi_wl['victory'].mean()

0.5

In [136]:
test = rikishi_wl.groupby(['wins','losses'])

In [144]:
print_all(test['victory'].mean())

wins  losses
0     0         0.500000
      1         0.467391
      2         0.479167
      3         0.380000
      4         0.406250
      5         0.368421
      6         0.454545
      7         0.000000
      8         0.500000
      9         0.666667
      10        0.000000
      11        1.000000
1     0         0.537634
      1         0.500000
      2         0.473684
      3         0.428571
      4         0.393939
      5         0.391304
      6         0.593750
      7         0.307692
      8         0.416667
      9         0.666667
      10        0.666667
      11        0.000000
      12        0.500000
      13        0.000000
2     0         0.520408
      1         0.500000
      2         0.496183
      3         0.423423
      4         0.457447
      5         0.483333
      6         0.360000
      7         0.416667
      8         0.461538
      9         0.550000
      10        0.636364
      11        0.250000
      12        0.500000
3     0     

In [134]:
records = test['victory'].()

In [135]:
records

wins
0     777
1     770
2     779
3     756
4     652
5     557
6     389
7     293
8     252
9     112
10     71
11     48
12     17
13     10
14      5
Name: victory, dtype: int64

In [122]:
def print_all(dataframe):
    pd.set_option('display.max_rows', None)
    print(dataframe)
    pd.set_option('display.max_rows', 30)

In [126]:
print_all(records)

wins  losses
0     0         0.500000
      1         0.467391
      2         0.479167
      3         0.380000
      4         0.406250
      5         0.368421
      6         0.454545
      7         0.000000
      8         0.500000
      9         0.666667
      10        0.000000
      11        1.000000
1     0         0.537634
      1         0.500000
      2         0.473684
      3         0.428571
      4         0.393939
      5         0.391304
      6         0.593750
      7         0.307692
      8         0.416667
      9         0.666667
      10        0.666667
      11        0.000000
      12        0.500000
      13        0.000000
2     0         0.520408
      1         0.500000
      2         0.496183
      3         0.423423
      4         0.457447
      5         0.483333
      6         0.360000
      7         0.416667
      8         0.461538
      9         0.550000
      10        0.636364
      11        0.250000
      12        0.500000
3     0     

In [124]:
records

wins  losses
0     0         0.500000
      1         0.467391
      2         0.479167
      3         0.380000
      4         0.406250
                  ...   
12    2         0.800000
13    0         1.000000
      1         0.714286
14    0         0.666667
      1         0.500000
Name: victory, Length: 120, dtype: float64